# Gráficos de Violín: Congruencia vs Incongruencia Ideológica

Este notebook genera **gráficos de violín** para visualizar las distribuciones de Cambio de Opinión (CO) y Cambio de Tiempo (CT) según congruencia ideológica.

## Objetivo:

Visualizar la distribución completa de las variables de congruencia/incongruencia (no solo las medias), permitiendo identificar:
- Asimetría de las distribuciones
- Presencia de múltiples modas
- Outliers y valores extremos
- Diferencias en dispersión entre grupos

## Variables analizadas:

- **CO_Congruente**: Ítems Progresistas → Izquierda + Ítems Conservadores → Derecha
- **CO_Incongruente**: Ítems Progresistas → Derecha + Ítems Conservadores → Izquierda
- **CT_Congruente**: Tiempos en dirección ideológicamente consistente
- **CT_Incongruente**: Tiempos en dirección ideológicamente inconsistente

## Elecciones:

- Generales
- Ballotage

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
sns.set_context('notebook', font_scale=1.1)

print('✓ Librerías cargadas exitosamente')

## 1. Cargar Datos

In [ ]:
# Rutas a los archivos
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Procesados')
Archivo_Generales = os.path.join(Ruta_Base, 'Generales_con_Congruencia.xlsx')
Archivo_Ballotage = os.path.join(Ruta_Base, 'Ballotage_con_Congruencia.xlsx')

# Cargar datos
df_Generales = pd.read_excel(Archivo_Generales)
df_Ballotage = pd.read_excel(Archivo_Ballotage)

print(f'✓ Datos cargados:')
print(f'  - Generales: {len(df_Generales)} registros')
print(f'  - Ballotage: {len(df_Ballotage)} registros')

# Verificar variables
vars_necesarias = ['CO_Congruente', 'CO_Incongruente', 'CT_Congruente', 'CT_Incongruente']

for nombre, df in [('Generales', df_Generales), ('Ballotage', df_Ballotage)]:
    faltantes = [v for v in vars_necesarias if v not in df.columns]
    if faltantes:
        print(f'  ⚠️  {nombre}: Faltan variables {faltantes}')
    else:
        print(f'  ✓ {nombre}: Todas las variables presentes')

## 2. Preparar Datos para Visualización

In [ ]:
def Crear_Grafico_Violin(df_long, variable_nombre, eleccion_nombre, guardar=True):
    """
    Crea gráfico de violín para una variable específica usando seaborn.
    
    Parámetros:
    -----------
    df_long : DataFrame en formato largo
    variable_nombre : 'Cambio de Opinión' o 'Cambio de Tiempo'
    eleccion_nombre : 'Generales' o 'Ballotage'
    guardar : bool, si True guarda el gráfico
    """
    
    # Filtrar datos
    df_plot = df_long[df_long['Variable'] == variable_nombre].copy()
    
    # Crear figura más grande
    fig, ax = plt.subplots(figsize=(12, 9))
    
    # Paleta de colores: Verde menta y Lavanda
    palette = {
        'Congruente': '#8dd3c7',
        'Incongruente': '#bebada'
    }
    
    # Crear gráfico de violín con seaborn
    sns.violinplot(
        data=df_plot,
        x='Tipo',
        y='Valor',
        palette=palette,
        cut=0,  # No extender más allá de los datos reales
        inner='box',  # Mostrar cuartiles con cajita
        linewidth=2,
        alpha=0.85,
        ax=ax
    )
    
    # Configurar fondo con grid sutil
    ax.set_facecolor('#fafafa')
    ax.grid(True, alpha=0.25, linestyle='-', linewidth=0.5, color='gray')
    ax.set_axisbelow(True)
    
    # Configurar ejes
    ax.set_xlabel('Tipo de Congruencia', fontsize=14, fontweight='bold')
    ax.set_ylabel('Valor', fontsize=14, fontweight='bold')
    ax.tick_params(axis='both', labelsize=12)
    
    # Título
    tipo_corto = 'CO' if variable_nombre == 'Cambio de Opinión' else 'CT'
    ax.set_title(f'{variable_nombre}: Congruente vs Incongruente\n{eleccion_nombre}',
                 fontsize=16, fontweight='bold', pad=20)
    
    # Añadir línea en cero
    ax.axhline(y=0, color='red', linestyle='--', linewidth=1.5, alpha=0.6, zorder=0)
    
    # Crear leyenda fuera del gráfico
    from matplotlib.patches import Patch
    from matplotlib.lines import Line2D
    
    legend_elements = [
        Patch(facecolor='#8dd3c7', edgecolor='black', linewidth=1.5, 
              label='Congruente', alpha=0.85),
        Patch(facecolor='#bebada', edgecolor='black', linewidth=1.5, 
              label='Incongruente', alpha=0.85),
        Line2D([0], [0], color='black', linewidth=6, label='Mediana'),
        Line2D([0], [0], color='black', linewidth=2, label='Cuartiles (Q1-Q3)'),
        Line2D([0], [0], color='red', linestyle='--', linewidth=1.5, 
               label='Línea de cero', alpha=0.6)
    ]
    
    # Posicionar leyenda fuera del área de ploteo
    ax.legend(handles=legend_elements, 
              loc='upper left', 
              bbox_to_anchor=(1.02, 1),
              fontsize=11,
              frameon=True,
              fancybox=True,
              shadow=True,
              framealpha=0.95)
    
    # Ajustar layout para que la leyenda no se corte
    plt.tight_layout()
    
    # Guardar como SVG
    if guardar:
        carpeta = 'Graficos_Violin'
        if not os.path.exists(carpeta):
            os.makedirs(carpeta)
        
        nombre_archivo = f'Violin_{tipo_corto}_{eleccion_nombre}.svg'
        ruta = os.path.join(carpeta, nombre_archivo)
        plt.savefig(ruta, format='svg', bbox_inches='tight', facecolor='white')
        print(f'✓ Gráfico guardado: {ruta}')
    
    plt.show()
    
    return fig, ax

In [ ]:
# Transformar datos a formato largo para seaborn
def preparar_datos_long(df):
    """
    Transforma datos de formato ancho a formato largo para visualización.
    """
    df_long_list = []
    
    # CO: Cambio de Opinión
    df_co = pd.DataFrame({
        'Valor': pd.concat([df['CO_Congruente'], df['CO_Incongruente']]),
        'Tipo': ['Congruente'] * len(df) + ['Incongruente'] * len(df),
        'Variable': 'Cambio de Opinión'
    })
    
    # CT: Cambio de Tiempo
    df_ct = pd.DataFrame({
        'Valor': pd.concat([df['CT_Congruente'], df['CT_Incongruente']]),
        'Tipo': ['Congruente'] * len(df) + ['Incongruente'] * len(df),
        'Variable': 'Cambio de Tiempo'
    })
    
    # Combinar
    df_long = pd.concat([df_co, df_ct], ignore_index=True)
    df_long = df_long.dropna(subset=['Valor'])
    
    return df_long

# Crear datasets en formato largo
df_gen_long = preparar_datos_long(df_Generales)
df_bal_long = preparar_datos_long(df_Ballotage)

print('✓ Datos transformados a formato largo:')
print(f'  - Generales: {len(df_gen_long)} observaciones')
print(f'  - Ballotage: {len(df_bal_long)} observaciones')
print(f'\nColumnas: {list(df_gen_long.columns)}')
print(f'\nPrimeras filas:')
print(df_gen_long.head(10))

## 3. Estadísticas Descriptivas

In [ ]:
print('='*70)
print('ESTADÍSTICAS DESCRIPTIVAS POR GRUPO')
print('='*70)

for nombre, df_long in [('Generales', df_gen_long), ('Ballotage', df_bal_long)]:
    print(f'\n📊 {nombre}:')
    print('-'*70)
    
    for variable in ['Cambio de Opinión', 'Cambio de Tiempo']:
        print(f'\n  {variable}:')
        
        df_var = df_long[df_long['Variable'] == variable]
        
        for tipo in ['Congruente', 'Incongruente']:
            datos = df_var[df_var['Tipo'] == tipo]['Valor']
            
            print(f'\n    {tipo}:')
            print(f'      n     = {len(datos)}')
            print(f'      Media = {datos.mean():.4f}')
            print(f'      Mediana = {datos.median():.4f}')
            print(f'      DE    = {datos.std():.4f}')
            print(f'      Min   = {datos.min():.4f}')
            print(f'      Max   = {datos.max():.4f}')
            print(f'      Q25   = {datos.quantile(0.25):.4f}')
            print(f'      Q75   = {datos.quantile(0.75):.4f}')

print('\n' + '='*70)

## 4. Gráficos de Violín

### 4.1. Gráficos: Generales

In [ ]:
print('Generando gráficos para Generales...\n')

# CO Generales
fig_co_gen, ax_co_gen = Crear_Grafico_Violin(
    df_gen_long,
    'Cambio de Opinión',
    'Generales'
)

In [ ]:
# CT Generales
fig_ct_gen, ax_ct_gen = Crear_Grafico_Violin(
    df_gen_long,
    'Cambio de Tiempo',
    'Generales'
)

### 4.2. Gráficos: Ballotage

In [ ]:
print('Generando gráficos para Ballotage...\n')

# CO Ballotage
fig_co_bal, ax_co_bal = Crear_Grafico_Violin(
    df_bal_long,
    'Cambio de Opinión',
    'Ballotage'
)

In [ ]:
# CT Ballotage
fig_ct_bal, ax_ct_bal = Crear_Grafico_Violin(
    df_bal_long,
    'Cambio de Tiempo',
    'Ballotage'
)

## 5. Pruebas de Significancia Estadística

In [ ]:
print('='*70)
print('TEST DE WILCOXON PAREADO')
print('='*70)
print('\nH₀: No hay diferencia entre Congruente e Incongruente')
print('H₁: Hay diferencia entre Congruente e Incongruente\n')

resultados_tests = []

for nombre_df, df in [('Generales', df_Generales), ('Ballotage', df_Ballotage)]:
    print(f'\n📊 {nombre_df}:')
    print('-'*70)
    
    for var_tipo, var_cong, var_incong in [
        ('CO', 'CO_Congruente', 'CO_Incongruente'),
        ('CT', 'CT_Congruente', 'CT_Incongruente')
    ]:
        datos_pareados = df[[var_cong, var_incong]].dropna()
        
        if len(datos_pareados) > 0:
            stat, p_valor = stats.wilcoxon(
                datos_pareados[var_cong],
                datos_pareados[var_incong]
            )
            
            # Determinar significancia
            if p_valor < 0.001:
                sig = '***'
            elif p_valor < 0.01:
                sig = '**'
            elif p_valor < 0.05:
                sig = '*'
            else:
                sig = 'ns'
            
            media_cong = datos_pareados[var_cong].mean()
            media_incong = datos_pareados[var_incong].mean()
            
            print(f'\n  {var_tipo}:')
            print(f'    n = {len(datos_pareados)}')
            print(f'    Media Congruente:    {media_cong:8.4f}')
            print(f'    Media Incongruente:  {media_incong:8.4f}')
            print(f'    Diferencia:          {media_cong - media_incong:8.4f}')
            print(f'    Estadístico W:       {stat:8.2f}')
            print(f'    p-valor:             {p_valor:.6f}')
            print(f'    Significancia:       {sig}')
            
            if sig != 'ns':
                if media_cong > media_incong:
                    print(f'    ✓ Mayor en CONGRUENTE')
                else:
                    print(f'    ✓ Mayor en INCONGRUENTE')
            
            resultados_tests.append({
                'Eleccion': nombre_df,
                'Variable': var_tipo,
                'n': len(datos_pareados),
                'Media_Congruente': media_cong,
                'Media_Incongruente': media_incong,
                'W': stat,
                'p_valor': p_valor,
                'Significancia': sig
            })

print('\n' + '='*70)

# Crear DataFrame con resultados
df_resultados = pd.DataFrame(resultados_tests)
print('\n📋 Resumen de Resultados:\n')
print(df_resultados.to_string(index=False))

## 6. Análisis de Distribuciones

In [ ]:
print('='*70)
print('ANÁLISIS DE FORMA DE DISTRIBUCIONES')
print('='*70)

for nombre, df in [('Generales', df_Generales), ('Ballotage', df_Ballotage)]:
    print(f'\n📊 {nombre}:')
    print('-'*70)
    
    for var_nombre, var_col in [
        ('CO_Congruente', 'CO_Congruente'),
        ('CO_Incongruente', 'CO_Incongruente'),
        ('CT_Congruente', 'CT_Congruente'),
        ('CT_Incongruente', 'CT_Incongruente')
    ]:
        datos = df[var_col].dropna()
        
        if len(datos) > 3:
            # Calcular asimetría (skewness) y curtosis
            skewness = stats.skew(datos)
            kurtosis = stats.kurtosis(datos)
            
            print(f'\n  {var_nombre}:')
            print(f'    Asimetría (skewness): {skewness:6.3f}', end='')
            
            if abs(skewness) < 0.5:
                print(' (aproximadamente simétrica)')
            elif skewness > 0:
                print(' (sesgada a la derecha)')
            else:
                print(' (sesgada a la izquierda)')
            
            print(f'    Curtosis:             {kurtosis:6.3f}', end='')
            
            if abs(kurtosis) < 1:
                print(' (mesocúrtica, similar a normal)')
            elif kurtosis > 0:
                print(' (leptocúrtica, más puntiaguda)')
            else:
                print(' (platicúrtica, más aplanada)')
            
            # Test de normalidad (Shapiro-Wilk para n < 5000)
            if len(datos) < 5000:
                w_stat, p_normal = stats.shapiro(datos)
                print(f'    Test Shapiro-Wilk:    p = {p_normal:.6f}', end='')
                
                if p_normal < 0.05:
                    print(' (NO normal)')
                else:
                    print(' (aproximadamente normal)')

print('\n' + '='*70)

## 7. Guardar Resumen de Resultados

In [ ]:
# Guardar tabla de resultados
Carpeta_Salida = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_Violin')
if not os.path.exists(Carpeta_Salida):
    os.makedirs(Carpeta_Salida)

archivo = os.path.join(Carpeta_Salida, 'Resultados_Tests_Congruencia.xlsx')
df_resultados.to_excel(archivo, index=False)
print(f'✓ Resultados guardados: {archivo}')

## 8. Resumen Final

In [ ]:
print('='*70)
print('RESUMEN: GRÁFICOS DE VIOLÍN - CONGRUENCIA IDEOLÓGICA')
print('='*70)

print('\n📊 Análisis completado:')
print('  - Gráficos generados: 4 (2 por elección)')
print('  - Variables analizadas: CO y CT (Congruente vs Incongruente)')
print('  - Elecciones: Generales y Ballotage')

print('\n📁 Archivos generados:')
print('  Gráficos:')
print('    - Graficos_Violin/Violin_CO_Generales.svg')
print('    - Graficos_Violin/Violin_CT_Generales.svg')
print('    - Graficos_Violin/Violin_CO_Ballotage.svg')
print('    - Graficos_Violin/Violin_CT_Ballotage.svg')
print('  Datos:')
print('    - Data/Resultados_Violin/Resultados_Tests_Congruencia.xlsx')

print('\n💡 Interpretación de gráficos de violín:')
print('  - Ancho del violín: densidad de observaciones en ese valor')
print('  - Línea horizontal (mediana): divide la distribución en 2 mitades iguales')
print('  - Diamante (media): centro de gravedad de la distribución')
print('  - Forma simétrica: distribución balanceada')
print('  - Múltiples "barrigas": posible distribución multimodal')

print('\n🎯 Hallazgos principales:')

for _, row in df_resultados.iterrows():
    if row['Significancia'] != 'ns':
        direccion = 'Congruente > Incongruente' if row['Media_Congruente'] > row['Media_Incongruente'] else 'Incongruente > Congruente'
        print(f"  {row['Eleccion']} - {row['Variable']}: {direccion} (p = {row['p_valor']:.4f} {row['Significancia']})")

print('\n' + '='*70)
print('✓ ANÁLISIS COMPLETADO')
print('='*70)